# V2

In [1]:
%matplotlib inline
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

#import statsmodels.api as sm
from datetime import datetime

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [2]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

In [3]:
X_test   = pd.read_csv("X_test.csv")

# CUSTOMERS

In [4]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,14089083.0,SE,1979-02-05 00:00:00,Femme,2013-03-16 23:00:05
1,12862066.0,FR,1982-08-04 00:00:00,Femme,2012-02-14 17:47:33
2,14791699.0,FR,1965-04-02 00:00:00,Femme,2013-10-04 23:10:42
3,10794664.0,FR,1966-04-09 00:00:00,Femme,2010-03-25 18:46:59
4,15268576.0,ES,1980-04-22 00:00:00,Femme,2014-03-19 10:48:39


### BirthDate -> Age

In [5]:
# Since dates aren't all in the same format, we take x[:19] only first 19 characters, which are we interested in
customers['BirthDate'] = customers['BirthDate'].apply(lambda x : datetime.strptime(x[:19], '%Y-%m-%d %H:%M:%S') )
customers['Age'] = 2016 - customers['BirthDate'].apply(lambda x : x.year)

In [6]:
for i in range(10,60,5):
    customers['Age' + str(i) + '-' + str(i+5)] = ((customers['Age']>i) & (customers['Age']<=i+5)).astype('float')

In [7]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60
0,14089083.0,SE,1979-02-05 00:00:00,Femme,2013-03-16 23:00:05,39,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,12862066.0,FR,1982-08-04 00:00:00,Femme,2012-02-14 17:47:33,36,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,14791699.0,FR,1965-04-02 00:00:00,Femme,2013-10-04 23:10:42,53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10794664.0,FR,1966-04-09 00:00:00,Femme,2010-03-25 18:46:59,52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,15268576.0,ES,1980-04-22 00:00:00,Femme,2014-03-19 10:48:39,38,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Gender -> Binaire

In [8]:
## 1 si une femme
customers['Sexe'] = customers['Gender'].eq('Femme').astype('float')

### FirstOrderDate -> Time in Year since first Order

In [9]:
customers['FirstOrderDate'] = customers['FirstOrderDate'].apply(lambda x : datetime.strptime(x[:19], '%Y-%m-%d %H:%M:%S') )
customers['FirstOrder'] = 2016 - customers['FirstOrderDate'].apply(lambda x : x.year)

In [10]:
for i in range(1,10):
    customers['FirstOrder' + '-' + str(i) + 'Y'] = (customers['FirstOrder']==i).astype('float')
customers['FirstOrder-10Y+'] = (customers['FirstOrder']>=10).astype('float')

In [11]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont été ajoutées à la DataFrame' %(len(Liste)))

In [12]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,...,FirstOrder-1Y,FirstOrder-2Y,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,14089083.0,SE,1979-02-05 00:00:00,Femme,2013-03-16 23:00:05,39,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12862066.0,FR,1982-08-04 00:00:00,Femme,2012-02-14 17:47:33,36,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14791699.0,FR,1965-04-02 00:00:00,Femme,2013-10-04 23:10:42,53,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10794664.0,FR,1966-04-09 00:00:00,Femme,2010-03-25 18:46:59,52,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,15268576.0,ES,1980-04-22 00:00:00,Femme,2014-03-19 10:48:39,38,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
customers.dtypes

CustomerId                float64
CountryISOCode             object
BirthDate                  object
Gender                     object
FirstOrderDate     datetime64[ns]
Age                         int64
Age10-15                  float64
Age15-20                  float64
Age20-25                  float64
Age25-30                  float64
Age30-35                  float64
Age35-40                  float64
Age40-45                  float64
Age45-50                  float64
Age50-55                  float64
Age55-60                  float64
Sexe                      float64
FirstOrder                  int64
FirstOrder-1Y             float64
FirstOrder-2Y             float64
FirstOrder-3Y             float64
FirstOrder-4Y             float64
FirstOrder-5Y             float64
FirstOrder-6Y             float64
FirstOrder-7Y             float64
FirstOrder-8Y             float64
FirstOrder-9Y             float64
FirstOrder-10Y+           float64
dtype: object

####  Delete columns

In [14]:
del customers['Gender']
del customers['FirstOrderDate']
del customers['FirstOrder']
del customers['BirthDate']
del customers['Age']

In [15]:
customers = customers.fillna(0)

In [16]:
customers.head()

,CustomerId,CountryISOCode,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,...,FirstOrder-1Y,FirstOrder-2Y,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,14089083.0,SE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12862066.0,FR,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14791699.0,FR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10794664.0,FR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,15268576.0,ES,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Products

In [17]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,Prenez une taille en dessous de votre pointure...
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,Prenez votre pointure habituelle
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,Prenez votre pointure habituelle
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle


#### 1) Fonction qui crée les colonnes nécessaires

In [18]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont été ajoutées à la DataFrame' %(len(Liste)))

#### 2) Analyse de la table

In [19]:
products.dtypes

VariantId                float64
GenderLabel               object
MarketTargetLabel         object
SeasonLabel               object
SeasonalityLabel          object
BrandId                  float64
UniverseLabel             object
TypeBrand                 object
ProductId                float64
ProductType               object
SupplierColor             object
ProductColorId           float64
MinSize                  float64
MaxSize                  float64
CalfTurn                 float64
UpperHeight              float64
HeelHeight               float64
PurchasePriceHT          float64
IsNewCollection          float64
SubtypeLabel              object
UpperMaterialLabel        object
LiningMaterialLabel       object
OutSoleMaterialLabel      object
RemovableSole             object
SizeAdviceDescription     object
dtype: object

#### 3) Modification+ajout de colonnes

##### * Gender Label

In [20]:
Label = [['Femme'],['Enfant'],['Homme'],['Sacs'],['Accessoires']]
creation(products,'GenderLabel',Label)

'5 colonne(s) ont été ajoutées à la DataFrame'

##### *MarketTargetLabel

In [21]:
Label2 = [['Contemporaine','contemporain'],['Suiveur'],['Fashionistas'],['Classique'],['Confort'],['Baroudeur','Aventurier'],['Skate']]
creation(products,'MarketTargetLabel',Label2)


'7 colonne(s) ont été ajoutées à la DataFrame'

##### *SeasonLabel

In [22]:
Label3 = products['SeasonLabel'].eq('Automne/Hiver').astype('float')
products['SeasonLabel'] = Label3

##### *SeasonalityLabel

In [23]:
Label4 = [['Saisonnier'],['Reconduit'],['Permanent']]
creation(products,'SeasonalityLabel',Label4)

'3 colonne(s) ont été ajoutées à la DataFrame'

##### *BrandId

In [24]:
## Problème car représenté par des nombres donc ordonés mais il y en a trop

##### *UniverseLabel

In [25]:
Label5 = [['tente'],['Ville'],['Sport'],['Green'],['Luxe'],['Confort'],['Soir']]
creation(products,'UniverseLabel',Label5)

'7 colonne(s) ont été ajoutées à la DataFrame'

##### *TypeBrand

In [26]:
Label6 = [['Standard'],['Luxe'],['Super']]
creation(products,'TypeBrand',Label6)

'3 colonne(s) ont été ajoutées à la DataFrame'

##### *ProductType

In [27]:
Label7 = [['Baskets'],['Sandales'],['Bottines'],['Escarpin'],['lacets'],['Ballerines'],['sport'],['Bottes']\
          ,['Mocass'],['Tongs'],['Chaussons'],['Mules'],['scratch'],['Sacs'],['Espadrilles'],['Maroquinerie']]
creation(products,'ProductType',Label7)

'16 colonne(s) ont été ajoutées à la DataFrame'

##### *SupplierColor

In [28]:
Label8 = [['Rouge','Red','Bordeaux','rouge','red'],['Jaune','Yellow','jaune','yellow'],\
          ['Vert','vert','green','Green'],['Marine','Bleu','BlUE','Navy','bleu','blue'],\
          ['Noir','Black','Nero','noir','black','nero','Schwarz','NOIR'],\
          ['Blanc','Incolore','Beige','blanc','incolor','beige'],\
          ['Gris','Grey','Anthracite','gris','grey','anthracite'],\
          ['Marron','Brown','Taupe','Tan','Cognac','Chocolate','marron','brown','taupe','tan','cognac'],\
          ['Rose','Pink','rose','pink','purple','Purple']]

Label9 = [['Dark','dark','Fonc','fonc'],['Light','light','clair','Clair']]

creation(products,'SupplierColor',Label8)
creation(products,'SupplierColor',Label9)

'2 colonne(s) ont été ajoutées à la DataFrame'

##### *ProductColorId

In [29]:
# Faut le del y'a au maximum 20 commande pour un seul ProductColorId

##### *MinSize

In [30]:
Label10 = [35,36,37,38,39,40,41,42,43,200]
for i in (Label10):
    products['MinSize' + str(i)] = (products['MinSize'].eq(i)).astype('float')

products['MinSize' + '18' + '-' + '34'] = ((products['MinSize']>=18) & (products['MinSize']<=34)).astype('float')
products['MinSize' + '44' + '-' + '48'] = ((products['MinSize']>=44) & (products['MinSize']<=48)).astype('float')

##### *MaxSize

In [31]:
Label11 = [35,36,37,38,39,40,41,42,43,44,45,200]
for i in (Label11):
    products['MaxSize' + str(i)] = (products['MaxSize'].eq(i)).astype('float')

products['MaxSize' + '18' + '-' + '34'] = ((products['MaxSize']>=18) & (products['MaxSize']<=34)).astype('float')
products['MaxSize' + '46' + '-' + '48'] = ((products['MaxSize']>=46) & (products['MaxSize']<=48)).astype('float')

##### *CalfTurn

In [32]:
# Soit il y en a un soit non
Label12 = products['CalfTurn'].eq(0).astype('float')
products['CalfTurn'] = Label12

##### *UpperHeight

In [33]:
Label13 = [0,2,5,6,7,8,9,10,30]
for i in (Label13):
    products['UpperHeight' + str(i)] = (products['UpperHeight'].eq(i)).astype('float')

products['UpperHeight' + '31' + '-' + '38'] = ((products['UpperHeight']>=31) & (products['UpperHeight']<=38)).astype('float')

##### *HeelHeight

In [34]:
Label14 = [0,1,2,3,4,5,6,7,8,9,10]
for i in (Label14):
    products['HeelHeight' + str(i)] = (products['HeelHeight'].eq(i)).astype('float')

products['HeelHeightsup' + '11'] = ((products['HeelHeight']>=11)).astype('float')

##### *PurchasePriceHT

In [35]:
# Pas besoin de changer, c'est une relation d'ordre

##### *IsNewCollection

In [36]:
# Déja en Binaire

##### *SubtypeLabel

In [37]:
Label15 = [['rond'],['Basses'],['Montantes'],['scratch'],['lacet'],['bride'],['zip'],\
           ['pointu'],['compens'],['carr'],['ouvert'],['lastiques'],['Derbies'],['pluie'],\
           ['Bateau'],['Salom'],['Fourr'],['aiguille'],['boots'],['Spartiates'],['Richelieus'],\
           ['Babies']]
creation(products,'SubtypeLabel',Label15)

'22 colonne(s) ont été ajoutées à la DataFrame'

##### *UpperMaterialLabel

In [38]:
Label16 = [['Cuir'],['Nubuck'],['Textile'],['verni'],['Synth'],['Caoutchouc']]
creation(products,'UpperMaterialLabel',Label16)

'6 colonne(s) ont été ajoutées à la DataFrame'

##### *LiningMaterialLabel

In [39]:
Label17 = [['Textile'],['Cuir'],['Synth'],['doubl']]
creation(products,'LiningMaterialLabel',Label16)

'6 colonne(s) ont été ajoutées à la DataFrame'

##### *OutSoleMaterialLabel

In [40]:
Label18 = [['Gomme'],['Caoutchouc'],['Synth'],['Cuir']]
creation(products,'OutSoleMaterialLabel',Label17)

'4 colonne(s) ont été ajoutées à la DataFrame'

##### *RemovableSole

In [41]:
Label19 = products['RemovableSole'].eq(True).astype('float')
products['RemovableSole'] = Label19

In [42]:
products['RemovableSole'].value_counts()

0.0    444714
1.0     88806
Name: RemovableSole, dtype: int64

##### *SizeAdviceDescription

In [43]:
Label20 = [['chausse normalement','Prenez votre pointure habituelle'],['dessous'],['dessus'],['larges'],['fins']]
creation(products,'SizeAdviceDescription',Label20)

'5 colonne(s) ont été ajoutées à la DataFrame'

#### 4) Delete columns

In [44]:
#del products['VariantId'] # Necessaire pour merge orders/products
del products['GenderLabel']
del products['MarketTargetLabel']
del products['SeasonalityLabel']
del products ['UniverseLabel']
del products ['TypeBrand']
del products ['ProductId'] #environ 600 par productId
del products['ProductType']
del products['SupplierColor']
del products['ProductColorId'] #max 20 par productIdcolor
del products['MinSize']
del products['MaxSize']
del products['CalfTurn']
del products['UpperHeight']
del products['HeelHeight']
del products['SubtypeLabel']
del products['UpperMaterialLabel']
del products['LiningMaterialLabel']
del products['OutSoleMaterialLabel']
del products['SizeAdviceDescription']

# del products['BrandId']  # peut être a supprimer essayer de le traiter

In [45]:
products = products.fillna(0)

In [46]:
products.head()

,VariantId,SeasonLabel,BrandId,PurchasePriceHT,IsNewCollection,RemovableSole,Femme,Enfant,Homme,Sacs,...,Textile,verni,Synth,Caoutchouc,doubl,Prenez votre pointure habituelle,dessous,dessus,larges,fins
0,728257.0,1.0,66.0,30.50,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,806356.0,1.0,842.0,43.00,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,768790.0,1.0,988.0,54.90,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,515679.0,1.0,769.0,34.50,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1025246.0,1.0,1244.0,43.76,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# ORDERS
### We need a function to process orders since it will have to be applied to both X_test and X_train

In [47]:
orders = X_train

In [48]:
def processOrders(orders):
    
    # LineItem to binary
    for i in range(1,5):
        orders['LineItem' + '_' + str(i) ] = (orders['LineItem']==i).astype('float')
    orders['LineItem_5+'] = (orders['LineItem']>=5).astype('float')
    
    # OrderStatusLabel to binary (expedie = 1)
    orders['OrderStatus'] = orders['OrderStatusLabel'].eq('Expédié').astype('float')
    
    # OrderTypeLabel to binary
    orders['OrderType'] = orders['OrderTypelabel'].eq('DIRECT').astype('float')
    
    # SeasonLabel to binary
    orders['Season'] = orders['SeasonLabel'].eq('Automne/Hiver').astype('float')
    
    # PayementModeLabel to binary
    Label = [['Carte','maestro'],['PayPal'],['Gratuit'],['iDeal'],['que'],['Bank'],['DHL']]
    creation(orders,'PayementModeLabel',Label)
    
    # CustomerTypeLabel to binary (fidelise = 1)
    orders['CustomerType'] = orders['CustomerTypeLabel'].eq('Fidélisé').astype('float')
    
    # DeviceTypeLabel to binary (attention, ND majoritaire)
    Label2 = [['Desktop'],['Tablet'],['Mobile']]
    creation(orders,'DeviceTypeLabel',Label2)
    
    # PricingTypeLabel to binary
    Label3 = [['Plein'],['Soldes'],['Avec CP'],['Sans CP'],['Vente Priv']]
    creation(orders,'PricingTypeLabel',Label3)
    
    # TotalLineItems = as LineItem
    for i in range(1,6):
        orders['TotalLineItems' + '_' + str(i) ] = (orders['TotalLineItems']==i).astype('float')
    orders['TotalLineItems_6+'] = (orders['TotalLineItems']>=6).astype('float')
    
    # OrderCreationDate -> Categories based on order creation hour
    orders['OrderCreationDate'] = orders['OrderCreationDate'].apply(lambda x : datetime.strptime(x[:19], '%Y-%m-%d %H:%M:%S') )
    temp = orders['OrderCreationDate'].apply(lambda x : x.hour)
    for i in range(0, 21, 3):
        orders['Hour_'+str(i)+'H_'+str((i+3))+'H'] = ((temp>=i) & (temp<(i+3))).astype('float')

    # Difference in days between OrderCreationDate and OrderShippingDate
    orders['OrderShipDate'] = orders['OrderShipDate'].apply(lambda x : datetime.strptime(x[:19], '%Y-%m-%d %H:%M:%S') )
    orders['CreationToShipDays'] = (orders['OrderCreationDate']-orders['OrderShipDate']).astype('timedelta64[D]')
    
    for i in range(1,10):
        orders['CreationToShip_'+str(i)+'_Day'] = (orders['CreationToShipDays']==i).astype('float')
    orders['CreationToShip_>10_Days'] = (orders['CreationToShipDays']>10).astype('float')
    
    # Delete columns 
    #del orders['OrderNumber']
    #del orders['VariantId'] # Necssaire pour merge orders/products
    #del orders['LineItem']
    #del orders['CustomerId'] # Necessaire pour merge orders/customers
    del orders['OrderStatusLabel']
    del orders['OrderTypelabel']
    del orders['SeasonLabel']
    del orders['PayementModeLabel']
    del orders['CustomerTypeLabel']
    del orders['DeviceTypeLabel']
    del orders['PricingTypeLabel']
    del orders['TotalLineItems']
    del orders['OrderCreationDate']
    del orders['OrderShipDate']
    del orders['CreationToShipDays']
    del orders['BillingPostalCode']
    # del orders['OrderNumCustomer']  # Le garder et voir statistiques dessus, pas traité.
    
    # Fill nulls
    orders = orders.fillna(0)

    return orders

In [49]:
orders = processOrders(orders)
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,...,CreationToShip_1_Day,CreationToShip_2_Day,CreationToShip_3_Day,CreationToShip_4_Day,CreationToShip_5_Day,CreationToShip_6_Day,CreationToShip_7_Day,CreationToShip_8_Day,CreationToShip_9_Day,CreationToShip_>10_Days
0,73521754,439729,1,12443972,FR,1,"52,64",1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,FR,1,"52,64",1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,FR,1,"13,17",1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,FR,1,"56,4",1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,FR,1,37,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
orders.columns

Index(['OrderNumber', 'VariantId', 'LineItem', 'CustomerId', 'IsoCode',
       'Quantity', 'UnitPMPEUR', 'OrderNumCustomer', 'IsOnSale', 'LineItem_1',
       'LineItem_2', 'LineItem_3', 'LineItem_4', 'LineItem_5+', 'OrderStatus',
       'OrderType', 'Season', 'maestro', 'PayPal', 'Gratuit', 'iDeal', 'que',
       'Bank', 'DHL', 'CustomerType', 'Desktop', 'Tablet', 'Mobile', 'Plein',
       'Soldes', 'Avec CP', 'Sans CP', 'Vente Priv', 'TotalLineItems_1',
       'TotalLineItems_2', 'TotalLineItems_3', 'TotalLineItems_4',
       'TotalLineItems_5', 'TotalLineItems_6+', 'Hour_0H_3H', 'Hour_3H_6H',
       'Hour_6H_9H', 'Hour_9H_12H', 'Hour_12H_15H', 'Hour_15H_18H',
       'Hour_18H_21H', 'CreationToShip_1_Day', 'CreationToShip_2_Day',
       'CreationToShip_3_Day', 'CreationToShip_4_Day', 'CreationToShip_5_Day',
       'CreationToShip_6_Day', 'CreationToShip_7_Day', 'CreationToShip_8_Day',
       'CreationToShip_9_Day', 'CreationToShip_>10_Days'],
      dtype='object')

# MERGING

In [51]:
def merge(orders, products, customers):
    orders_products = pd.merge(orders,products,how='inner',on='VariantId')
    data = pd.merge(orders_products,customers,how='inner',on='CustomerId')
    del data['OrderNumber']
    del data['VariantId']
    del data['CustomerId']
    del data['LineItem']
    # Pour tester une premiere fois on supprime ces colonnes
    #del X_data['IsoCode']
    #del X_data['CountryISOCode']
    #del X_data['BrandId']
    
    return data

### Merge X_train

In [ ]:
X_train = merge(orders, products, customers)

# Preprocessing Test set

In [ ]:
X_test = processOrders(X_test)
X_test = merge(X_test, products, customers)

# Defining a feature transformation

In [ ]:
def funk_mask(d,masked_features):
    " Defining a simple mask over the input data "
    columns_ext = masked_features
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    print("1")
    g = lambda x: x.replace(",",".")
    print("2")
    X1.UnitPMPEUR = map(np.float64,(map(g,X1.UnitPMPEUR)))
    print("3")
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    print("4")
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    print("5")
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    print("6")
    res = pd.concat([X1,X2],axis=1)
    print("7")
    res = res.fillna(0)
    return(res)

# Applying the mask

In [ ]:
features = []

In [ ]:
x1 = funk_mask(X_train,features)
x2 = funk_mask(X_test,features)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

# Standardizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(x1)

# Apply transform to both the training set and the test set.
x1 = scaler.transform(x1)
x2 = scaler.transform(x2)

# PCA

In [ ]:
# Make an instance of the Model
pca = PCA(.95)

# Fit PCA on training set
pca.fit(x1)

# Apply the mapping (transform) to both the training set and the test set
x1 = pca.transform(x1)
x2 = pca.transform(x2)

In [ ]:
x1

In [ ]:
x2

# Supervised learning : Logistic regression model

In [ ]:
clf = RandomForestClassifier()
clf.fit(x1[:100000], data.ReturnQuantityBin[:100000])
y_tosubmit = clf.predict_proba(x2)
score_forest = np.mean(cross_val_score(clf,x1[:100000], data.ReturnQuantityBin[:100000],cv=10))
print(score_forest)

In [ ]:
print(y_tosubmit[:,1].shape)
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

# Score of our prediction : on the train

In [ ]:
yres = clf.predict_proba(x1[:100001])
print(roc_auc_score(data.ReturnQuantityBin.iloc[:100001],yres[:,1]))

# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')